In [1]:
import pandas as pd
import ipywidgets as ipw
import numpy as np
import networkx as nx

('HP:0000219: Thin upper lip vermilion',
 'HP:0000343: Long philtrum',
 'HP:0000369: Low-set ears',
 'HP:0000453: Choanal atresia',
 'HP:0000494: Downslanted palpebral fissures',
 'HP:0000821: Hypothyroidism',
 'HP:0000824: Growth hormone deficiency',
 'HP:0000871: Panhypopituitarism',
 'HP:0001162: Postaxial hand polydactyly',
 'HP:0001233: 2-3 finger syndactyly',
 'HP:0001360: Holoprosencephaly',
 'HP:0010626: Anterior pituitary agenesis',
 'HP:0011220: Prominent forehead',
 'HP:0011755: Ectopic posterior pituitary',
 'HP:0040075: Hypopituitarism')
 
 GLI2
WNT2
TP53

FZD8
HDAC7
NPHP4
ATM
OR7A5
OR6C1
OR1B1
OR2A25
OR7G3
ZNF717
WDR34
IL4I1
DPAGT1
SLC18A1
ASXL1
DFFB
SLC35D3
RYR3
MLH3
FAM21C
SCAF11
ABI3BP
EFCAB13
HMCN1
PCLO
WBP11
CNST
GDPD4
CCDC66
PRR4
GPLD1
NLRP2
RETNLB
FICD
TBC1D5
 

In [2]:
#simple helper functions
def gmean(data, axis=0):
    return np.exp(np.mean(np.log(data), axis=axis))

def gstd(data, axis=0):
    mu = gmean(data, axis=axis)
    return np.exp(np.sqrt(np.mean((np.log(data) - mu)**2, axis=axis)))

def combine_ranks(data, hpo_terms):
    # get geometric mean of ranks for given terms
    hpo_terms = [x for x in hpo_terms if x in data.columns] # remove terms not in dataset    
    return pd.Series(gmean(data.loc[:, hpo_terms], axis=1), index=data.index)

def reduce_terms(data, hpo_tree, hpo_terms):
    #given a list of hpo terms, get the closest ancestor in dataset columns
    term_list = set()
    for term in hpo_terms:
        #get the list of terms that actually are in dataset
        path = [x for x in nx.shortest_path(hpo_tree, term, 'HP:0000118') if x in data.columns]
        if len(path) > 0:
            term_list.add(path[0])
    return term_list

def random_ranks(ranked_genes, input_terms, n_iter=1000):
    r_hpo = np.array([x for x in hpo_net.nodes() if nx.has_path(hpo_net, x, 'HP:0000118')])
    random_sum = np.zeros((n_iter, len(ranked_genes)))
    for n in range(n_iter):
        np.random.shuffle(r_hpo)
        random_terms = r_hpo[:len(input_terms)]
        ranked_random = combine_ranks(subset_ranks, reduce_terms(subset_ranks, hpo_net, random_terms))
        random_sum[n] = ranked_random
    return pd.Series(gmean(random_sum), index=ranked_genes.index)

def build_table(df):
    spool = """
    <table width='100%'>
    <tr>
    <th>Entrez ID</th>
    <th>Gene Symbol</th>
    <th>Name</th>
    <th>Model Entropy</th>
    <th>Model Quality</th>
    <th>Score</th>
    </tr>"""
    n_hp = model_ranks.shape[1] * 1.
#    rrq = random_ranks(df, input_terms)
    for gene in df.index:
        g_name = gene_annotation.node[gene]['name']
        g_symbol = list(gene_annotation.neighbors(gene))[0]
        model_q = np.sum(model_ranks.loc[gene] < df.loc[gene]) / n_hp
        if df.loc[gene] < 0.1:
            default_color = "#8EFA00"
        elif df.loc[gene] < 0.2:
            default_color = "#FFD479"
        else:
            default_color = "#FF7E79"
        spool += '<tr><td>%s</td><td>%s</td><td>%s</td><td align="center">%.4f</td><td align="center">%.4f</td><td bgcolor="%s" align="center">%.4f</td></tr>' % (gene, g_symbol, g_name, 0.0,model_q, default_color, df.loc[gene])
    spool += "</table>"
    return spool

def get_valid_genes(gene_string):
    # first retain only genes that are in our annotation
    genes = gene_string.split()
    final_genes = set()
    genes_in = [x for x in genes if x in gene_annotation.nodes()]
    genes_out = [x for x in genes if not x in gene_annotation.nodes()]
    for g in genes_in:
        if gene_annotation.node[g]['type'] == 'entrez':
            final_genes.add(g)
        else:
            #a gene symbol is given
            [final_genes.add(x) for x in gene_annotation.neighbors(g)]
    return final_genes, genes_out

def add_warning_excl(genes_out):
    spool = '<hr><div class="excluded"><b>Warning!</b> The following items were excluded from the analysis:<br>'
    for g in genes_out:
        spool += '%s<br>' % g
    spool += "</div>"
    return spool

In [3]:
#load data
#github does not allow files larger than 100 MB, so I splitted our model
model_ranks = pd.read_table("data/global_ind_pheno_tumor_munge.best_pred.00.csv.gz", sep=',', index_col=0, compression='gzip')
for chunk in range(1, 11):
    model_ranks = pd.concat([model_ranks, pd.read_table("data/global_ind_pheno_tumor_munge.best_pred.%02d.csv.gz" % chunk, sep=',', index_col=0, compression='gzip')])
model_ranks.index = [str(x) for x in model_ranks.index]



In [4]:
    
#scored terms with some annotation
hp_annot = pd.read_table("data/global_ind_pheno_tumor_munge.best.csv", sep=',', index_col=0)
#combined ranks for OMIM diseases
disease_net = nx.read_graphml("data/disease_annot.graphml")
#the HPO obo tree
hpo_net = nx.read_graphml("data/hp.180127.obo.graphml")
#gene annotation to have gene symbol and/or entrez ID
gene_annotation = nx.read_graphml("data/gene_annotation.graphml")

In [6]:
hpo_ids = [x for x in hpo_net.nodes() if nx.has_path(hpo_net, x, 'HP:0000118')]
hpo_ids.sort()
hpo_ids = hpo_ids[1:]
hpo_names = [hpo_net.node[x]['name'] for x in hpo_ids]
hpo_terms = ["%s: %s" % (hpo_ids[x], hpo_names[x]) for x in range(len(hpo_ids))]
disease_ids = [x for x in disease_net.nodes() if disease_net.node[x]['type'] == 'disease' and 'name' in disease_net.node[x]]
disease_ids.sort()
disease_names = [disease_net.node[x]['name'] for x in disease_ids if 'name' in disease_net.node[x]]
diseases = ["%s: %s" % (disease_ids[x], disease_names[x]) for x in range(len(disease_ids))]
selected_terms = set()

#define the widget for HPO search
#the elements
gene_list = ipw.Textarea(
    placeholder='Enter your gene list',
    description='Genes:',
    disabled=False
)

search_hp_widget = ipw.Text(placeholder='Start typing phenotypes...') #to search the phenotypes
search_dis_widget = ipw.Text(placeholder='Start typing description...') #to search the phenotypes
options_widget = ipw.SelectMultiple(options=hpo_terms) #this lists all terms to be selected
disease_widget = ipw.Select(options=diseases)
add_hp_button = ipw.Button(description='Add term')
remove_button = ipw.Button(description='Remove term')
add_dis_button = ipw.Button(description='Add disease')
selected_widget = ipw.SelectMultiple()
submit_button = ipw.Button(description='Submit')
reset_button = ipw.Button(description='Reset')
out = ipw.Output()
results = ipw.HTML()

#stitch together
hp_search = ipw.VBox([search_hp_widget, options_widget, add_hp_button])
dis_search = ipw.VBox([search_dis_widget, disease_widget, add_dis_button])
chosen_area = ipw.VBox([selected_widget, remove_button])
bottom_area = ipw.HBox([reset_button, submit_button])
selection_area = ipw.HBox([hp_search, ipw.HTML(value="<b> OR </b>"), dis_search])
multi_select = ipw.VBox([selection_area, chosen_area])

#define actions
def on_hp_search(change):
    search_input = change['new']
    if search_input == '':
        # Reset search field
        new_options = hpo_terms
    else:
        # Filter by search field 
        new_options = [x for x in hpo_terms if search_input.lower() in x.lower()]
    options_widget.options = new_options

def on_dis_search(change):
    search_input = change['new']
    if search_input == '':
        # Reset search field
        new_options = diseases
    else:
        # Filter by search field 
        new_options = [x for x in diseases if search_input.lower() in x.lower()]
    disease_widget.options = new_options

    
def on_add_hp(b):
    new_options = set([x for x in selected_widget.options] + [x for x in options_widget.value])
    new_options = list(new_options)
    new_options.sort()
    selected_widget.options = new_options

def on_add_dis(b):
    dis_id = disease_widget.value[:11]
    hp_map = [x for x in disease_net.neighbors(dis_id) if nx.has_path(hpo_net, x, 'HP:0000118')]
    hp_names = [hpo_net.node[x]['name'] for x in hp_map]
    hp_terms = ["%s: %s" % (hp_map[x], hp_names[x]) for x in range(len(hp_map))]
    new_options = set([x for x in selected_widget.options] + hp_terms)
    new_options = list(new_options)
    new_options.sort()
    selected_widget.options = new_options

    
    
def on_remove(b):
    v = [x for x in selected_widget.options if not x in selected_widget.value]
    selected_widget.options = v

def on_submit(b):
    input_genes = []
    input_terms = []
    excluded_genes = []
    if not gene_list.value:
        output = "<b>You should provide some genes</b>"
    else:
        input_genes, excluded_genes = get_valid_genes(gene_list.value)
    if not selected_widget.options:
        output = "<b>You should provide some HPO terms</b>"
    input_terms = [x[:10] for x in selected_widget.options]
    if not input_genes:
        output = "<b>None of the genes provided is valid</b>"
    elif input_terms:
        #we have genes and terms
        subset_ranks = model_ranks.loc[input_genes] #subset data to perform less calculation
        ranked_genes = combine_ranks(subset_ranks, reduce_terms(subset_ranks, hpo_net, input_terms)).sort_values()
        output = build_table(ranked_genes)
    if excluded_genes:
        output += add_warning_excl(excluded_genes)
    results.value = output


def on_reset(b):
    search_hp_widget.value = ''
    search_dis_widget.value = ''
    gene_list.value = ''
    selected_widget.options = ()
    results.value = ''

#link actions
add_hp_button.on_click(on_add_hp)
add_dis_button.on_click(on_add_dis)                      
remove_button.on_click(on_remove)    
submit_button.on_click(on_submit)
reset_button.on_click(on_reset)
search_hp_widget.observe(on_hp_search, names='value')
search_dis_widget.observe(on_dis_search, names='value')
#display



## DISCAN notebook
This is a simple implementation of [DISCAN](https://doi.org/10.1101/120121) using Jupyter interactive notebook. 

This web application is intended to score a gene list against a given set of phenotypes encoded in the [Human Phenotype Ontology](http://human-phenotype-ontology.github.io). Phenotypes can be added by selecting a specifc disease from the disease menu.

---
**Warning**

*If you are using this notebook on [binder](http://mybinder.org), you may expect poor performances*

----

#### 1) Enter a list of genes here

In [7]:
display(gene_list)

Textarea(value='', description='Genes:', placeholder='Enter your gene list')

#### 2) Choose some phenotypes

In [8]:
display(multi_select)

VBox(children=(HBox(children=(VBox(children=(Text(value='', placeholder='Start typing phenotypes...'), SelectMultiple(options=('HP:0000003: Multicystic kidney dysplasia', 'HP:0000008: Abnormality of female internal genitalia', 'HP:0000009: Functional abnormality of the bladder', 'HP:0000010: Recurrent urinary tract infections', 'HP:0000011: Neurogenic bladder', 'HP:0000012: Urinary urgency', 'HP:0000013: Hypoplasia of the uterus', 'HP:0000014: Abnormality of the bladder', 'HP:0000015: Bladder diverticulum', 'HP:0000016: Urinary retention', 'HP:0000017: Nocturia', 'HP:0000019: Urinary hesitancy', 'HP:0000020: Urinary incontinence', 'HP:0000021: Megacystis', 'HP:0000022: Abnormality of male internal genitalia', 'HP:0000023: Inguinal hernia', 'HP:0000024: Prostatitis', 'HP:0000025: Functional abnormality of male internal genitalia', 'HP:0000026: Male hypogonadism', 'HP:0000027: Azoospermia', 'HP:0000028: Cryptorchidism', 'HP:0000029: Testicular atrophy', 'HP:0000030: Testicular gonadoblastoma', 'HP:0000031: Epididymitis', 'HP:0000032: Abnormality of male external genitalia', 'HP:0000033: Ambiguous genitalia, male', 'HP:0000034: Hydrocele testis', 'HP:0000035: Abnormality of the testis', 'HP:0000036: Abnormality of the penis', 'HP:0000037: Male pseudohermaphroditism', 'HP:0000039: Epispadias', 'HP:0000040: Long penis', 'HP:0000041: Chordee', 'HP:0000042: Absent external genitalia', 'HP:0000044: Hypogonadotrophic hypogonadism', 'HP:0000045: Abnormality of the scrotum', 'HP:0000046: Scrotal hypoplasia', 'HP:0000047: Hypospadias', 'HP:0000048: Bifid scrotum', 'HP:0000049: Shawl scrotum', 'HP:0000050: Hypoplastic male external genitalia', 'HP:0000051: Perineal hypospadias', 'HP:0000052: Urethral atresia, male', 'HP:0000053: Macroorchidism', 'HP:0000054: Micropenis', 'HP:0000055: Abnormality of female external genitalia', 'HP:0000056: Abnormality of the clitoris', 'HP:0000058: Abnormality of the labia', 'HP:0000059: Hypoplastic labia majora', 'HP:0000060: Clitoral hypoplasia', 'HP:0000061: Ambiguous genitalia, female', 'HP:0000062: Ambiguous genitalia', 'HP:0000063: Fused labia minora', 'HP:0000064: Hypoplastic labia minora', 'HP:0000065: Labial hypertrophy', 'HP:0000066: Labial hypoplasia', 'HP:0000067: Urethral atresia, female', 'HP:0000068: Urethral atresia', 'HP:0000069: Abnormality of the ureter', 'HP:0000070: Ureterocele', 'HP:0000071: Ureteral stenosis', 'HP:0000072: Hydroureter', 'HP:0000073: Ureteral duplication', 'HP:0000074: Ureteropelvic junction obstruction', 'HP:0000075: Renal duplication', 'HP:0000076: Vesicoureteral reflux', 'HP:0000077: Abnormality of the kidney', 'HP:0000078: Abnormality of the genital system', 'HP:0000079: Abnormality of the urinary system', 'HP:0000080: Abnormality of reproductive system physiology', 'HP:0000081: Duplicated collecting system', 'HP:0000083: Renal insufficiency', 'HP:0000085: Horseshoe kidney', 'HP:0000086: Ectopic kidney', 'HP:0000089: Renal hypoplasia', 'HP:0000090: Nephronophthisis', 'HP:0000091: Abnormality of the renal tubule', 'HP:0000092: Tubular atrophy', 'HP:0000093: Proteinuria', 'HP:0000095: Abnormality of renal glomerulus morphology', 'HP:0000096: Glomerulosclerosis', 'HP:0000097: Focal segmental glomerulosclerosis', 'HP:0000098: Tall stature', 'HP:0000099: Glomerulonephritis', 'HP:0000100: Nephrotic syndrome', 'HP:0000103: Polyuria', 'HP:0000104: Renal agenesis', 'HP:0000105: Enlarged kidney', 'HP:0000107: Renal cyst', 'HP:0000108: Renal corticomedullary cysts', 'HP:0000110: Renal dysplasia', 'HP:0000111: Renal juxtaglomerular cell hypertrophy/hyperplasia', 'HP:0000112: Nephropathy', 'HP:0000113: Polycystic kidney dysplasia', 'HP:0000114: Proximal tubulopathy', 'HP:0000117: Renal phosphate wasting', 'HP:0000118: Phenotypic abnormality', 'HP:0000119: Abnormality of the genitourinary system', 'HP:0000121: Nephrocalcinosis', 'HP:0000122: Unilateral renal agenesis', 'HP:0000123: Nephritis', 'HP:0000124: Renal tubular dysfunction', 'HP:0000125: Pelvic kidney', 'HP:0000126: Hyd

#### 3) Submit your query and check results

In [10]:
display(bottom_area, results)

HBox(children=(Button(description='Reset', style=ButtonStyle()), Button(description='Submit', style=ButtonStyle())))

HTML(value='')

----
<small>
Davide Cittaro<br>
[Center for Translational Genomics and Bioinformatics, Milano](http://www.hsr.it/research/organization/divisions-centers/center-for-translational-genomics-and-bioinformatics/)
</small>

In [8]:
options_widget.value = [x for x in hpo_terms if x[:10] in 'HP:0000219;HP:0000343;HP:0000369;HP:0000453;HP:0000494;HP:0000821;HP:0000824;HP:0000871;HP:0001162;HP:0001233;HP:0001360;HP:0010626;HP:0011220;HP:0011755;HP:0040075'.split(';')]

In [16]:
input_genes, excluded_genes = get_valid_genes(gene_list.value)
input_terms = [x[:10] for x in selected_widget.options]

In [17]:
len(input_genes)

37

In [18]:
subset_ranks = model_ranks.loc[input_genes] #subset data to perform less calculation
ranked_genes = combine_ranks(subset_ranks, reduce_terms(subset_ranks, hpo_net, input_terms)).sort_values()

In [21]:
q = pd.Series([np.sum(model_ranks.loc[x] < ranked_genes.loc[x]) / 1368  for x in ranked_genes.index], index=ranked_genes.index)

In [22]:
import scipy.stats

In [26]:
results_basis = model_ranks.loc[input_genes, reduce_terms(subset_ranks, hpo_net, input_terms)]

In [51]:
ee = scipy.stats.entropy(results_basis.values.T) / np.log(results_basis.shape[1])
te = scipy.stats.entropy(subset_ranks.values.T) / np.log(subset_ranks.shape[1])

In [52]:
ee = pd.Series(ee, index=results_basis.index)
te = pd.Series(te, index=subset_ranks.index)
ss = gstd(results_basis.T)

In [54]:
pd.concat([ee, ranked_genes, te, ss], axis=1).sort_values(1)

,0,1,2,3
7273,0.903283,0.000138,0.638291,7456.260008
27445,0.932703,0.000939,0.656814,1107.644082
7157,0.582428,0.001154,0.698427,1373.231431
83872,0.685801,0.002390,0.657334,478.363613
6263,0.630175,0.004374,0.692129,272.341764
472,0.655405,0.025250,0.777807,53.164052
2822,0.841806,0.046856,0.934501,26.736823
51729,0.873043,0.078476,0.954192,16.186204
27030,0.891854,0.081109,0.942109,15.261008
2736,0.976426,0.084416,0.928356,13.437770


In [ ]:
mr = gmean(xxx, axis=1)
ar = gmean(model_ranks.loc[input_genes], axis=1)


In [ ]:
np.argmin(ar)


In [ ]:
ar = model_ranks.loc[mr.index, 'HP:0000118']

In [ ]:
(mr / ar).sort_values()

In [ ]:
gstd(xxx.T).loc[ranked_genes.index]

In [ ]:
gstd(xxx.T, axis=0)

In [ ]:
ee = pd.Series(ee, index=ranked_genes.index).loc[ranked_genes.index]

In [ ]:
ee

In [ ]:
len(input_genes)

In [ ]:
ee.shape, ss.shape

In [ ]:
%pylab
scatter(ee, ss)

In [ ]:
X = []
for x in range(100):
    r = np.random.rand(15)
    X.append([scipy.stats.entropy(np.log(r))/np.log(15),np.std(np.log(r))])
X = np.array(X)

In [ ]:
 scipy.stats.entropy(np.log(np.ones(15) - 0.1)) / np.log(15)

In [ ]:
scatter(X[:, 0], X[:, 1])

In [ ]:
data = xxx

In [ ]:
mu = gmean(data, axis=1)

In [ ]:
mu.sort_values()

In [ ]:
(np.log(data) - mu.ravel())**2

In [ ]:
pd.Series(np.exp(np.sqrt(np.mean((np.log(np.array(data)) - np.array(mu)[:, None])**2, axis=1))), index=xxx.index).loc[ranked_genes.index]

In [ ]:
ss = gstd(xxx.T)

In [ ]:
ss

In [ ]:
options_widget.value

In [20]:
list(gene_annotation.neighbors('10'))

['NAT2']